In [1]:
import sys
import os
import random
import json
import gc
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from functools import partial
from albumentations import (Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, CenterCrop, 
                            HorizontalFlip, VerticalFlip, Rotate, ShiftScaleRotate, Transpose)
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from tensorflow import keras

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset

In [2]:
path = "../input/pgpdataset2022mar07/dataset/"
image_path = path+"test/"

IMAGE_SIZE = (512,512)
submission_df = pd.read_csv('../input/pgpdataset2022mar07/dataset/test.csv')

In [3]:
class CustomResNext(nn.Module):
        def __init__(self, model_name='resnext50_32x4d', pretrained=False):
            super().__init__()
            self.model = timm.create_model(model_name, pretrained=pretrained)
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, 7)

        def forward(self, x):
            x = self.model(x)
            return x

In [4]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_path_ID'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        image = Image.open(file_name)
        image = np.array(image)
        if image.shape[-1]>3: image = image[:, :, :-1]
        if image.shape[0]==1:
            image = np.ones((512, 512, 3))
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [5]:
used_models_pytorch = {"resnext": [f'../input/pgp-training/resnext50_32x4d_fold{fold}_best.pth' for fold in [0,1,2,3,4]]}

In [ ]:
if "resnext" in used_models_pytorch:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def get_transforms():
        return Compose([Resize(512, 512),
                        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                        ToTensorV2()])

    def inference(model, states, test_loader, device):
        model.to(device)

        probabilities = []
        for i, (images) in enumerate(test_loader):
            images = images.to(device)
            avg_preds = []
            for state in states:
                model.load_state_dict(state['model'])
                model.eval()
                with torch.no_grad():
                    y_preds = model(images)
                avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            avg_preds = np.mean(avg_preds, axis=0)
            probabilities.append(avg_preds)
        return np.concatenate(probabilities)
    

    predictions_resnext = pd.DataFrame(columns={"image_ID"})
    predictions_resnext["image_ID"] = submission_df["image_ID"].values
    predictions_resnext['image_path_ID'] = image_path + predictions_resnext['image_ID'].astype(str)

    model = CustomResNext('resnext50_32x4d', pretrained=False)
    states = [torch.load(f) for f in used_models_pytorch["resnext"]]

    test_dataset = TestDataset(predictions_resnext, transform=get_transforms())
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=1, pin_memory=True)
    predictions = inference(model, states, test_loader, device)

    predictions_resnext['resnext'] = [np.squeeze(p) for p in predictions]
    predictions_resnext = predictions_resnext.drop(["image_path_ID"], axis=1)
    

    torch.cuda.empty_cache()
    try:
        del(model)
        del(states)
    except:
        pass
    gc.collect()

In [ ]:
submission_df["label"] = 0

if "resnext" in used_models_pytorch:
    submission_df = submission_df.merge(predictions_resnext, on="image_ID")

In [ ]:
submission_df["label"] = submission_df.apply(lambda row: np.argmax(
        [np.sum(e) for e in zip(row["resnext"])]), axis=1)

In [ ]:
labels = {'label': {0: 'Cricket', 1: 'Wrestling', 2: 'Tennis', 3:'Badminton', 4: 'Soccer', 5: 'Swimming', 6: 'Karate'}}
submission_df = submission_df.replace(labels)

In [ ]:
submission_df[["image_ID","label"]].to_csv("submission.csv", index=False)